In [13]:
import numpy as np
import pandas as pd
import torch
import nltk
import string
import re
import scipy


In [2]:
data = pd.read_csv('data/train.csv.zip',compression='zip')

In [3]:
data.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [4]:
for col in ['toxic','obscene','threat','insult','identity_hate']:
    print(col,data[col].sum() )
print("total : {}".format(len(data)))

toxic 15294
obscene 8449
threat 478
insult 7877
identity_hate 1405
total : 159571


## First Attempt: 
Basic TF-IDF vectors with punctuation and stopwords removed

In [5]:
corpus = (data['comment_text'].values)
corpus = [re.sub('['+string.punctuation+']','', x) for x in corpus]
corpus = [x.replace('\n',' ') for x in corpus]
corpus = [x.replace('\t',' ') for x in corpus]

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
tfidf = TfidfVectorizer(tokenizer = nltk.word_tokenize, stop_words='english')

In [8]:
tfidf.fit(corpus)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function word_tokenize at 0x7f02d1836d90>, use_idf=True,
        vocabulary=None)

### Basic Logistic Regression
I'm going to fit a logistic regression on these basic tfidf vectors as a wayto get a baseline on what kind of accuracy we can expect. For now I am going to simply classify each comment as toxic or not toxic by combining the various kinds of toxic comments.

In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [22]:
def bad_comment(x):
    s = x['toxic']+x['obscene']+x['threat']+x['insult']+x['identity_hate']
    if s>0: return 1
    else: return 0
    
data['general_toxic'] = data.apply(bad_comment,axis=1) 

In [10]:
X = tfidf.transform(corpus)

In [23]:
y = data['general_toxic'].values

In [24]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [25]:
lin_classifier = LogisticRegression()

In [ ]:
lin_classifier.fit(X_train,y_train)